# IMPUTING VALUES FOR THE NAN'S


# visitor_hist_starrating:  94.92 %
After some testing it doesnt seem to matter if we impute the mean/median of all entries, or we train a model to predict the starrating, or we impute the mean of the people from the same country with the same destination. All the MSE's were the same.


# visitor_hist_adr_usd:  94.9 %
After some testing it doesnt seem to matter if we impute the mean/median of all entries, or we train a model to predict the history usd, or we impute the mean of the people from the same country with the same destination. All the MSE's were the same.


# prop_review_score:  0.15 %
If a hotel doesnt have a review score, the same hotel also doesnt have a review score in other searches. The correlation between prop_review_score and click_bool is highest when we impute 0 for the missing values.


# prop_location_score2:  21.99 %
It seems that the location_score2 is based on the search destination and the hotel location. It is probable that the further away the hotel is from the original search destination the lower the location_score2 is. Furthermore, after inspection of this attribute it seems that there exists a modest negative correlation between position and location_score2 (with filtering a filter on the random results) [r = -.20]. The hotels with missing values for location_score2 are ranked almost always at the bottom (position mean = 19, sd = 6). So i suggest we impute 0 for missing values of this attribute. This would also increase the significance between the location_score2 and the click_bool (i.e. the probability of a click is twice as unlikely when it doesnt have a location_score2)


# srch_query_affinity_score:  93.6 %
Some hotels with a nan value have an affinity score in another search. If we use the mean of the affinity scores from other searches it is significantly more accurate than the mean of all searches. When a hotel doesnt have an affinity score anywhere, the mean of the affinity scores in the same search destination is used. The RMSE of this procedure is 13.88 (SD = 31.36) while the RMSE of using the mean of everything is 16.07 (SD = 34.30).


# orig_destination_distance:  32.43 %
Either a search has the distance for every result or for none. Imputing the mean leads to a RMSE of 2000km (sd = 3300km), but imputing the mean of the people from the same country going to the same destination leads to a RMSE of 560km (sd = 800km)*. So i think that this would be a great solution to fill in the NA values for orig_destination_distance!

*if there were no people from the same country A going to the same destination in country B, i used the mean of the people from country A that go to country B (so less specific than the same destination within country B)

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('training_set_VU_DM_2014.csv')
df.shape

(4958347, 54)

In [ ]:
print('\n\tPercentage of missing values:')
for column in df.columns:
    percentage = round((len(df[df[column].isnull()])/len(df))*100,2)
    if percentage > 0:
        print(column + ': ', percentage, '%')

In [49]:
# First check the simplest solutions of imputing 1 value for all missing values in a column

from scipy.stats import pearsonr

for column in ['visitor_hist_starrating','visitor_hist_adr_usd', 'prop_review_score',
'prop_location_score2','srch_query_affinity_score','orig_destination_distance']:
    
    result = []
    df2 = df[[column, 'click_bool']]

    r1, p = pearsonr(df2[df2[column].notnull()][column], df2[df2[column].notnull()]['click_bool'])
    result.append((r1, 'old'))
    for option in ['mean', 'median', 'zero', 'firstq', 'thirdq']:
        df2 = df[[column, 'click_bool']]
        if option == 'mean':
            df2.loc[:,column] = df2[column].fillna(df2[column].mean())
            r, p = pearsonr(df2[column], df2['click_bool'])
        if option == 'median':
            df2.loc[:,column] = df2[column].fillna(df2[column].median())
            r, p = pearsonr(df2[column], df2['click_bool'])
        if option == 'zero':
            df2.loc[:,column] = df2[column].fillna(0)
            r, p = pearsonr(df2[column], df2['click_bool'])
        if option == 'firstq':
            df2.loc[:,column] = df2[column].fillna(df2[column].quantile(q = .25))
            r, p = pearsonr(df2[column], df2['click_bool'])
        if option == 'thirdq':
            df2.loc[:,column] = df2[column].fillna(df2[column].quantile(q = .75))
            r, p = pearsonr(df2[column], df2['click_bool'])
        result.append((r, option))
    best_corr, best_option = max(result)
    if best_option != 'old':
        print(f"{column} had an original correlation of r = {round(r1, 5)} with the click_bool. \nBest option: impute {best_option} with r = {round(best_corr, 5)}")
        print()
    else:
        print(f"{column} had an original correlation of {round(r1, 5)} with the click_bool. \nThere was no better alternative.")
        print()

C:\Users\Mick\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


visitor_hist_starrating had an original correlation of 0.00254 with the click_bool. 
There was no better alternative.

visitor_hist_adr_usd had an original correlation of 0.00386 with the click_bool. 
There was no better alternative.

prop_review_score had an original correlation of r = 0.02342 with the click_bool. 
Best option: impute zero with r = 0.02344

prop_location_score2 had an original correlation of r = 0.07381 with the click_bool. 
Best option: impute zero with r = 0.08309

srch_query_affinity_score had an original correlation of 0.04176 with the click_bool. 
There was no better alternative.

orig_destination_distance had an original correlation of 0.00229 with the click_bool. 
There was no better alternative.



# Visitor_hist_starrating

In [ ]:
import math
dfstar = df[df['visitor_hist_starrating'].notnull()][['srch_id', 'visitor_location_country_id', 'srch_destination_id', 'prop_country_id', 'visitor_hist_starrating']].drop_duplicates()
pred = list()
h = 0
for entry in dfstar.values:
    visitor_country_id = entry[1]
    srch_id = entry[0]
    srch_dest = entry[2]
    country = entry[3]
    same_loc_df = dfstar[(dfstar['visitor_location_country_id'] == visitor_country_id) & (dfstar['srch_id'] != srch_id) &(dfstar['srch_destination_id'] == srch_dest)]
    if len(same_loc_df) > 0:
        pred.append(same_loc_df['visitor_hist_starrating'].mean())
    else: # if there are no other searches from the same country with a history
        same_loc_df = dfstar[(dfstar['visitor_location_country_id'] == visitor_country_id) & (dfstar['srch_id'] != srch_id) &(dfstar['prop_country_id'] == country)]
        if len(same_loc_df) > 0:
            pred.append(same_loc_df['visitor_hist_starrating'].mean())
        else: # if there are no other searches from the same countro to the same country with a history
            h+=1
            pred.append(dfstar[dfstar['srch_id'] != srch_id]['visitor_hist_starrating'].mean())
pred = np.asarray(pred)

print('\tImpute mean of the people from the same country:')
print('MSE:', math.sqrt(((dfstar['visitor_hist_starrating'] - pred)**2).mean()))
print('SD:', math.sqrt(((dfstar['visitor_hist_starrating'] - pred)**2).std()))
print()
print('\tImpute mean of everyone:')
print('MSE:', math.sqrt(((dfstar['visitor_hist_starrating'] - dfstar['visitor_hist_starrating'].mean())**2).mean()))
print('SD:', math.sqrt(((dfstar['visitor_hist_starrating'] - dfstar['visitor_hist_starrating'].mean())**2).std()))
print()
print(f'{round(h/len(dfstar),2)*100} % of the instances had no other searches from the same country with a history.')

# Visitor_hist_adr_usd

In [ ]:
import math
dfstar = df[df['visitor_hist_adr_usd'].notnull()][['srch_id', 'visitor_location_country_id', 'srch_destination_id', 'prop_country_id', 'visitor_hist_adr_usd']].drop_duplicates()
pred = list()
i = 0
for entry in dfstar.values:
    visitor_country_id = entry[1]
    srch_id = entry[0]
    srch_dest = entry[2]
    country = entry[3]
    same_loc_df = dfstar[(dfstar['visitor_location_country_id'] == visitor_country_id) & (dfstar['srch_id'] != srch_id) &(dfstar['srch_destination_id'] == srch_dest)]
    if len(same_loc_df) > 0:
        pred.append(same_loc_df['visitor_hist_adr_usd'].mean())
    else: # if there are no other searches from the same country to the same destination with a history
        same_loc_df = dfstar[(dfstar['visitor_location_country_id'] == visitor_country_id) & (dfstar['srch_id'] != srch_id) &(dfstar['prop_country_id'] == country)]
        if len(same_loc_df) > 0:
            pred.append(same_loc_df['visitor_hist_adr_usd'].mean())
        else: # if there are no other searches from the same countro to the same country with a history
            i += 1
            pred.append(dfstar[dfstar['srch_id'] != srch_id]['visitor_hist_adr_usd'].mean())

pred = np.asarray(pred)

print('\tImpute mean of the people from the same country:')
print('MSE:', math.sqrt(((dfstar['visitor_hist_adr_usd'] - pred)**2).mean()))
print('SD:', math.sqrt(((dfstar['visitor_hist_adr_usd'] - pred)**2).std()))
print()
print('\tImpute mean of everyone:')
print('MSE:', math.sqrt(((dfstar['visitor_hist_adr_usd'] - dfstar['visitor_hist_adr_usd'].mean())**2).mean()))
print('SD:', math.sqrt(((dfstar['visitor_hist_adr_usd'] - dfstar['visitor_hist_adr_usd'].mean())**2).std()))
print()
print(f'{round(i/len(dfstar),2)*100} % of the instances had no other searches from the same country with a history.')

# Origin destination distance

In [ ]:
import math
dfstar = df[df['orig_destination_distance'].notnull()][['srch_id', 'visitor_location_country_id', 'srch_destination_id', 'prop_country_id', 'orig_destination_distance']].drop_duplicates()
pred = list()
h = 0
for entry in dfstar.values[::4000]:
    visitor_country_id = entry[1]
    srch_id = entry[0]
    srch_dest = entry[2]
    country = entry[3]
    same_loc_df = dfstar[(dfstar['visitor_location_country_id'] == visitor_country_id) & (dfstar['srch_destination_id'] == srch_dest)]
    if len(same_loc_df) > 0:
        pred.append(same_loc_df['orig_destination_distance'].mean())
    else: # if there are no other searches from the same country with a history
        same_loc_df = dfstar[(dfstar['visitor_location_country_id'] == visitor_country_id) & ( dfstar['prop_country_id'] == country)]
        if len(same_loc_df) > 0:
            pred.append(same_loc_df['orig_destination_distance'].mean())
        else: # if there are no other searches from the same country to the same country with a history
            h+=1
            pred.append(dfstar[dfstar['srch_id'] != srch_id]['orig_destination_distance'].mean())
pred = np.asarray(pred)

print('\tImpute mean of the people from the same country:')
print('MSE:', math.sqrt(((dfstar['orig_destination_distance'][::4000] - pred)**2).mean()))
print('SD:', math.sqrt(((dfstar['orig_destination_distance'][::4000] - pred)**2).std()))
print()
print('\tImpute mean of everyone:')
print('MSE:', math.sqrt(((dfstar['orig_destination_distance'][::4000] - dfstar['orig_destination_distance'][::4000].mean())**2).mean()))
print('SD:', math.sqrt(((dfstar['orig_destination_distance'][::4000] - dfstar['orig_destination_distance'][::4000].mean())**2).std()))
print()
print(f'{round(h/len(dfstar),2)*100} % of the instances had no other searches from the same country with a history.')

# srch_query_affinity_score

In [ ]:
# since there are a lot of hotels with nan values that dont have none values for some other searches it is faster
# to create a database with the mean affinity score for every hotel, with search destination
# WARNING: this may take a while to complete

df2 = df[['prop_id', 'srch_destination_id', 'srch_query_affinity_score']]

props = []
means = []
srch_destination = []
prop_id_set = set(df2[df2['srch_query_affinity_score'].notnull()]['prop_id'])
deci = int(len(prop_id_set)/10)
i = 0

for prop in prop_id_set:
    propdf = df2[(df2['prop_id'] == prop)]
    prop2df = propdf[propdf['srch_query_affinity_score'].notnull()]
    if len(prop2df) > 0:
        mean = prop2df['srch_query_affinity_score'].mean()
        srch_dest = propdf['srch_destination_id'].mode()
        if len(srch_dest) > 1:
            for dest in srch_dest:   
                props.append(prop)
                means.append(mean)
                srch_destination.append(dest)
        else:
            props.append(prop)
            means.append(mean)
            srch_destination.append(srch_dest[0])
    if len(props)%deci == 0:
        i += 1
        print(i*10, '%', end='\t')
    
database = pd.DataFrame()
database['prop_id'] = np.asarray(props)
database['srch_destination_id'] = np.asarray(srch_destination)
database['mean_srch_query_affinity_score'] = np.asarray(means)
database.to_csv('hotel_affinity_scores.csv')

In [35]:
import math
dfstar = df[df['srch_query_affinity_score'].notnull()][['prop_id', 'srch_destination_id', 'srch_query_affinity_score']].drop_duplicates()
pred = list()
h = 0
for entry in dfstar.values:
    prop_id = entry[0]
    srch_dest = entry[1]
    same_loc_df = database[(database['prop_id'] == prop_id)]
    if len(same_loc_df) > 0:
        pred.append(same_loc_df['mean_srch_query_affinity_score'].mean())
    else: # if the hotel doesnt have a affinity score anywhere
        same_loc_df = database[database['srch_destination_id'] == srch_dest]
        if len(same_loc_df) > 0:
            pred.append(same_loc_df['mean_srch_query_affinity_score'].mean())
        else:
            h += 1
            pred.append(database['mean_srch_query_affinity_score'].mean())
            
pred = np.asarray(pred)

print('\tImpute mean of the people from the same hotel:')
print('MSE:', math.sqrt(((dfstar['srch_query_affinity_score'] - pred)**2).mean()))
print('SD:', math.sqrt(((dfstar['srch_query_affinity_score'] - pred)**2).std()))
print()
print('\tImpute mean of everyone:')
print('MSE:', math.sqrt(((dfstar['srch_query_affinity_score'] - dfstar['srch_query_affinity_score'].mean())**2).mean()))
print('SD:', math.sqrt(((dfstar['srch_query_affinity_score'] - dfstar['srch_query_affinity_score'].mean())**2).std()))
print()
print(f'{round(h/len(dfstar),2)*100} % of the instances had no other searches from the same country with a history.')

	Impute mean of the people from the same country:
MSE: 13.882222527986682
SD: 31.365682804256604

	Impute mean of everyone:
MSE: 16.066876944759166
SD: 34.302345344899905

0.0 % of the instances had no other searches from the same country with a history.
